In [ ]:
'''
看代码差不多看懂了，我在这里大概解释一下吧：
soft triple loss其实就是把每一个class分成了k个center，在这里我们设k=10，通过将data分类到centers上而不是class上以增强分类的准确率，
个人感觉这里有点像boost之类的操作。。。soft triple的代码把训练分成两部分，一部分是backbone的神经网络，另一部分是一个全连接（dim，cN*K），
这里dim是backbone出来的数据维数（embedding的维度），cN是class number， K是center数量。

源代码里面backbone的训练用的是lr=0.0001的adam，而fc的训练用的是lr=0.01的adam，这里论文如果没有详细解释的话我个人认为有特意调参的痕迹

因为few shot的代码结构实在有点离谱，所以我只改了baseline+soft的网络，用soft triple的参数0.0001/0.01效果比baseline要差很多，你可以试试别的参数
参数的设置在io_utils中，当然你也可以用 --modellr和--centerlr修改
使用soft triple 的命令是 --method baseline_soft

'''

# io_utils

In [ ]:
def parse_args(script):
    parser.add_argument('--K'           , default=10,           help='num of centers for each class')
    parser.add_argument('--la'          , default=20, type=float,  help='lambda of soft triple')
    parser.add_argument('--gamma'       , default=0.1, type=float, help='gamma of soft triple')
    parser.add_argument('--tau'         , default=0.2, type=float,  help='tau of soft triple')
    parser.add_argument('--margin'      , default=0.01, type=float, help='margin of soft triple')
    parser.add_argument('--modellr', default=0.0001, type=float, help='model learning rate of soft triple')
    parser.add_argument('--centerlr', default=0.01, type=float, help='center learning rate of soft triple')

# train.py

In [ ]:
from methods.baselinetrain_softtriple import BaselineTrainSoft

if params.method == 'baseline_soft':
    optimizer = torch.optim.Adam([{"params": model.feature.parameters(), "lr": params.modellr},
                                      {"params": model.fc, "lr": params.centerlr}]) #line 28-30

elif params.method =='baseline_soft'
    model           = BaselineTrainSoft( model_dict[params.model], la = params.la, gamma = params.gamma, 
                                        tau=params.tau, margin=params.margin, K=params.K, cN=params.num_classes)# line115-116

# save_features.py

In [ ]:
from methods.baselinetrain_softtriple import BaselineTrainSoft #line 13

if not params.method in ['baseline', 'baseline++', 'baseline_soft'] : #line77

# test.py

In [ ]:
from methods.baselinetrain_softtriple import BaselineTrainSoft #line 19

    if params.method == ('baseline' or 'baseline_soft'):# line 61

# methods.baselinetrain_softtriple.py

In [ ]:
This is a new file.

In [44]:
m = 'b_l'

if (m =='b') or (m=='b_l'):
    print(1)

1


In [2]:
def train(base_loader, val_loader, model, optimization, start_epoch, stop_epoch, params):    
    if optimization == 'Adam':
        optimizer = torch.optim.Adam(model.parameters())
    else:
        raise ValueError('Unknown optimization, please define by yourself')

    max_acc = 0       

    for epoch in range(start_epoch,stop_epoch):
        model.train()
        model.train_loop(epoch, base_loader,  optimizer ) #model are called by reference, no need to return 
        model.eval()

        if not os.path.isdir(params.checkpoint_dir):
            os.makedirs(params.checkpoint_dir)

        acc = model.test_loop( val_loader)
        if acc > max_acc : #for baseline and baseline++, we don't use validation in default and we let acc = -1, but we allow options to validate with DB index
            print("best model! save...")
            max_acc = acc
            outfile = os.path.join(params.checkpoint_dir, 'best_model.tar')
            torch.save({'epoch':epoch, 'state':model.state_dict()}, outfile)

        if (epoch % params.save_freq==0) or (epoch==stop_epoch-1):
            outfile = os.path.join(params.checkpoint_dir, '{:d}.tar'.format(epoch))
            torch.save({'epoch':epoch, 'state':model.state_dict()}, outfile)

    return model

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim
import torch.optim.lr_scheduler as lr_scheduler
import time
import os
import glob

import configs
import backbone
from data.datamgr import SimpleDataManager, SetDataManager
from methods.baselinetrain import BaselineTrain
from methods.baselinefinetune import BaselineFinetune
from methods.protonet import ProtoNet
from methods.matchingnet import MatchingNet
from methods.relationnet import RelationNet
from methods.maml import MAML
#from methods.baselinetrain_softtriple import BaselineTrainSoft
from io_utils import model_dict, parse_args, get_resume_file  

In [12]:
import backbone
import utils
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import numpy as np
import torch.nn.functional as F
from torch.nn import init

class BaselineTrainSoft(nn.Module):
    def __init__(self, model_func, la, gamma, tau, margin, cN, K):
        super(BaselineTrainSoft, self).__init__()
        self.feature = model_func()
        self.la = la
        self.gamma = 1. / gamma
        self.tau = tau
        self.margin = margin
        self.cN = cN
        self.K = K
        self.fc = Parameter(torch.Tensor(self.feature.final_feat_dim, cN * K))
        self.weight = torch.zeros(cN * K, cN * K, dtype=torch.bool)
        for i in range(0, cN):
            for j in range(0, K):
                self.weight[i * K + j, i * K + j + 1:(i + 1) * K] = 1
        init.kaiming_uniform_(self.fc, a=math.sqrt(5))
        self.DBval = False;  # only set True for CUB dataset, see issue #31


    def forward(self, x):
        x = Variable(x)
        out = self.feature.forward(x)
        centers = F.normalize(self.fc, p=2, dim=0)
        simInd = out.matmul(centers)
        simStruc = simInd.reshape(-1, self.cN, self.K)
        prob = F.softmax(simStruc * self.gamma, dim=2)
        simClass = torch.sum(prob * simStruc, dim=2)

        return simClass


    def forward_loss(self, x, y):
        simClass = self.forward(x)
        y = Variable(y)
        marginM = torch.zeros(simClass.shape)
        marginM[torch.arange(0, marginM.shape[0]), y] = self.margin
        lossClassify = F.cross_entropy(self.la * (simClass - marginM), y)
        if self.tau > 0 and self.K > 1:
            simCenter = centers.t().matmul(centers)
            reg = torch.sum(torch.sqrt(2.0 + 1e-5 - 2. * simCenter[self.weight])) / (self.cN * self.K * (self.K - 1.))
            return lossClassify + self.tau * reg
        else:
            return lossClassify


    def train_loop(self, epoch, train_loader, optimizer):
        print_freq = 10
        avg_loss = 0


        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            loss = self.forward_loss(x, y)
            loss.backward()
            optimizer.step()

            avg_loss = avg_loss + loss.item()

            if i % print_freq == 0:
                # print(optimizer.state_dict()['param_groups'][0]['lr'])
                print('Epoch {:d} | Batch {:d}/{:d} | Loss {:f}'.format(epoch, i, len(train_loader),
                                                                        avg_loss / float(i + 1)))

    def test_loop(self, val_loader):
        if self.DBval:
            return self.analysis_loop(val_loader)
        else:
            return -1  # no validation, just save model during iteration

    def analysis_loop(self, val_loader, record=None):
        class_file = {}
        for i, (x, y) in enumerate(val_loader):
            x = x.cuda()
            x_var = Variable(x)
            feats = self.feature.forward(x_var).data.cpu().numpy()
            labels = y.cpu().numpy()
            for f, l in zip(feats, labels):
                if l not in class_file.keys():
                    class_file[l] = []
                class_file[l].append(f)

        for cl in class_file:
            class_file[cl] = np.array(class_file[cl])

        DB = DBindex(class_file)
        print('DB index = %4.2f' % (DB))
        return 1 / DB  # DB index: the lower the better


def DBindex(cl_data_file):
    # For the definition Davis Bouldin index (DBindex), see https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index
    # DB index present the intra-class variation of the data
    # As baseline/baseline++ do not train few-shot classifier in training, this is an alternative metric to evaluate the validation set
    # Emperically, this only works for CUB dataset but not for miniImagenet dataset

    class_list = cl_data_file.keys()
    cl_num = len(class_list)
    cl_means = []
    stds = []
    DBs = []
    for cl in class_list:
        cl_means.append(np.mean(cl_data_file[cl], axis=0))
        stds.append(np.sqrt(np.mean(np.sum(np.square(cl_data_file[cl] - cl_means[-1]), axis=1))))

    mu_i = np.tile(np.expand_dims(np.array(cl_means), axis=0), (len(class_list), 1, 1))
    mu_j = np.transpose(mu_i, (1, 0, 2))
    mdists = np.sqrt(np.sum(np.square(mu_i - mu_j), axis=2))

    for i in range(cl_num):
        DBs.append(np.max([(stds[i] + stds[j]) / mdists[i, j] for j in range(cl_num) if j != i]))
    return np.mean(DBs)

In [13]:
model           = BaselineTrainSoft( model_dict['Conv4'], la = 20, gamma = 0.1, tau=0.2, margin=0.01, K=10, cN=200)

In [42]:
model.fc

Parameter containing:
tensor([[ 0.0125,  0.0220,  0.0003,  ..., -0.0117, -0.0030, -0.0176],
        [ 0.0073, -0.0082,  0.0056,  ...,  0.0091,  0.0127,  0.0139],
        [ 0.0043,  0.0114, -0.0197,  ..., -0.0193, -0.0005, -0.0089],
        ...,
        [-0.0006, -0.0164, -0.0019,  ...,  0.0221, -0.0036, -0.0177],
        [ 0.0155,  0.0201, -0.0022,  ...,  0.0011,  0.0204, -0.0114],
        [-0.0004,  0.0104,  0.0202,  ...,  0.0049,  0.0021, -0.0048]],
       requires_grad=True)

In [33]:
model.parameters

<bound method Module.parameters of BaselineTrainSoft(
  (feature): ConvNet(
    (trunk): Sequential(
      (0): ConvBlock(
        (C): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (BN): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (trunk): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
      )
      (1): ConvBlock(
        (C): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (BN): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
   

In [39]:
set(model.state_dict())

{'fc',
 'feature.trunk.0.BN.bias',
 'feature.trunk.0.BN.num_batches_tracked',
 'feature.trunk.0.BN.running_mean',
 'feature.trunk.0.BN.running_var',
 'feature.trunk.0.BN.weight',
 'feature.trunk.0.C.bias',
 'feature.trunk.0.C.weight',
 'feature.trunk.0.trunk.0.bias',
 'feature.trunk.0.trunk.0.weight',
 'feature.trunk.0.trunk.1.bias',
 'feature.trunk.0.trunk.1.num_batches_tracked',
 'feature.trunk.0.trunk.1.running_mean',
 'feature.trunk.0.trunk.1.running_var',
 'feature.trunk.0.trunk.1.weight',
 'feature.trunk.1.BN.bias',
 'feature.trunk.1.BN.num_batches_tracked',
 'feature.trunk.1.BN.running_mean',
 'feature.trunk.1.BN.running_var',
 'feature.trunk.1.BN.weight',
 'feature.trunk.1.C.bias',
 'feature.trunk.1.C.weight',
 'feature.trunk.1.trunk.0.bias',
 'feature.trunk.1.trunk.0.weight',
 'feature.trunk.1.trunk.1.bias',
 'feature.trunk.1.trunk.1.num_batches_tracked',
 'feature.trunk.1.trunk.1.running_mean',
 'feature.trunk.1.trunk.1.running_var',
 'feature.trunk.1.trunk.1.weight',
 'featu

In [22]:
class SoftTriple(nn.Module):
    def __init__(self, la, gamma, tau, margin, dim, cN, K):
        super(SoftTriple, self).__init__()
        self.la = la
        self.gamma = 1./gamma
        self.tau = tau
        self.margin = margin
        self.cN = cN
        self.K = K
        self.fc = Parameter(torch.Tensor(dim, cN*K))
        self.weight = torch.zeros(cN*K, cN*K, dtype=torch.bool)
        for i in range(0, cN):
            for j in range(0, K):
                self.weight[i*K+j, i*K+j+1:(i+1)*K] = 1
        init.kaiming_uniform_(self.fc, a=math.sqrt(5))
        return

    def forward(self, input, target):
        centers = F.normalize(self.fc, p=2, dim=0)
        simInd = input.matmul(centers)
        simStruc = simInd.reshape(-1, self.cN, self.K)
        prob = F.softmax(simStruc*self.gamma, dim=2)
        simClass = torch.sum(prob*simStruc, dim=2)
        marginM = torch.zeros(simClass.shape)
        marginM[torch.arange(0, marginM.shape[0]), target] = self.margin
        lossClassify = F.cross_entropy(self.la*(simClass-marginM), target)
        if self.tau > 0 and self.K > 1:
            simCenter = centers.t().matmul(centers)
            reg = torch.sum(torch.sqrt(2.0+1e-5-2.*simCenter[self.weight]))/(self.cN*self.K*(self.K-1.))
            return lossClassify+self.tau*reg
        else:
            return lossClassify

In [23]:
model2 = SoftTriple(la = 20, gamma = 0.1, tau=0.2, margin=0.01, dim=1800, cN=200, K=10 )

In [40]:
model2.state_dict()

OrderedDict([('fc',
              tensor([[ 0.0061, -0.0220,  0.0207,  ..., -0.0223,  0.0048, -0.0074],
                      [-0.0124,  0.0143, -0.0033,  ..., -0.0067,  0.0220,  0.0181],
                      [-0.0003, -0.0049,  0.0126,  ..., -0.0083,  0.0203, -0.0079],
                      ...,
                      [ 0.0223, -0.0216,  0.0189,  ..., -0.0210,  0.0161,  0.0045],
                      [ 0.0209, -0.0169,  0.0019,  ..., -0.0138,  0.0115,  0.0037],
                      [ 0.0211,  0.0062, -0.0125,  ...,  0.0066,  0.0051,  0.0194]]))])

In [45]:
a = torch.randn(5,10)

In [48]:
b = F.softmax(a, dim=1)
b

tensor([[0.0264, 0.0443, 0.1060, 0.2311, 0.1519, 0.0253, 0.0555, 0.0742, 0.1948,
         0.0906],
        [0.1676, 0.0251, 0.1073, 0.0132, 0.1444, 0.0247, 0.1127, 0.1685, 0.0982,
         0.1383],
        [0.0637, 0.1627, 0.0166, 0.0180, 0.0240, 0.4840, 0.1047, 0.0611, 0.0180,
         0.0471],
        [0.0108, 0.1580, 0.0467, 0.0117, 0.0884, 0.0286, 0.0510, 0.2217, 0.1797,
         0.2034],
        [0.1166, 0.0602, 0.1243, 0.0632, 0.0205, 0.0162, 0.3391, 0.0831, 0.1275,
         0.0494]])